In [42]:
import pennylane as qml
from pennylane import numpy as np
from pennylane import qchem
from pennylane.qchem import excitations

In [38]:
# Define the atoms in the molecule
symbols = ["Li", "H"]

# Define the coordinates (in Ångströms)
coordinates = np.array([
    [0.0, 0.0, 0.0],     # Lithium atom at the origin
    [0.0, 0.0, 1.6]      # Hydrogen atom positioned 1.6 Å along the z-axis
])

In [39]:
hamiltonian, qubits = qchem.molecular_hamiltonian(symbols, coordinates, charge=0)
print(qubits)

12


In [44]:
electrons=4
hf = qchem.hf_state(electrons=electrons, orbitals=qubits)
print(hf)

[1 1 1 1 0 0 0 0 0 0 0 0]


In [41]:
num_wires = qubits
dev = qml.device("default.qubit", wires=num_wires)
@qml.qnode(dev)
def exp_energy(state):
    qml.BasisState(np.array(state), wires=range(num_wires))
    return qml.expval(hamiltonian)

exp_energy(hf)

tensor(-7.66194677, requires_grad=True)

In [64]:
# singles, doubles = excitations(4, qubits)
# print(f"Singles Excitations: {singles}")
# print(f"Doubles Excitations: {doubles}")

# def ansatz(params):
#     qml.BasisState(hf, wires=range(num_wires))
#     for i, excitation in enumerate(doubles):
#         qml.DoubleExcitation(params[i], wires=excitation)

def ansatz(params):
    qml.BasisState(hf, wires=range(num_wires))
    qml.DoubleExcitation(params[0], wires=[0, 1, 2, 3])
    qml.DoubleExcitation(params[1], wires=[0, 1, 4, 5])
    qml.DoubleExcitation(params[2], wires=[2, 3, 4, 5])


In [65]:
@qml.qnode(dev)
def cost_function(params):
    ansatz(params)
    return qml.expval(hamiltonian)

initial_params = np.zeros(len(doubles))
cost_function(initial_params)

tensor(-7.66194677, requires_grad=True)

In [67]:
# Initialize optimizer
opt = qml.GradientDescentOptimizer(stepsize=0.6)

# Initial parameters for 3 DoubleExcitation gates
theta = np.array([0.0, 0.0, 0.0], requires_grad=True)

# Initialize energy and parameter tracking lists
energy = [cost_function(theta)]
angle = [theta]
max_iterations = 20

# VQE optimization loop
for n in range(max_iterations):
    theta, prev_energy = opt.step_and_cost(cost_function, theta)
    energy.append(cost_function(theta))
    angle.append(theta)

    if n % 5 == 0:
        print(f"Step = {n}, Energy = {energy[-1]:.8f} Ha")


Step = 0, Energy = -7.66202722 Ha
Step = 5, Energy = -7.66212606 Ha
Step = 10, Energy = -7.66213582 Ha
Step = 15, Energy = -7.66213701 Ha


In [68]:
print(f"Final ground state energy: {energy[-1]:.8f} Ha")
print(f"Final parameters: {theta[0]:.8f}, {theta[1]:.8f}")

Final ground state energy: -7.66213715 Ha
Final parameters: 0.00000000, 0.00630248


In [69]:
@qml.qnode(dev)
def ground_state(params):
    ansatz(params)
    return qml.state()

ground_state(theta)

tensor([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
       shape=(4096,), requires_grad=True)